In [ ]:
from bs4 import BeautifulSoup
import requests

# Create a list of URLs for each page to scrape
pages = [requests.get('https://www.voursa.com/Index.cfm?PN='+str(j)+'&gct=1&sct=11&gv=13') for j in range(2, 27)]

urls = []

# Loop through each page to extract image URLs
for page in pages:
    soup = BeautifulSoup(page.text, 'html.parser')
    # Find all 'img' tags in the page
    img_tags = soup.find_all('img')
    # Extract the 'src' attribute from each 'img' tag
    img_urls = [img['src'] for img in img_tags if 'src' in img.attrs]
    # Add the extracted URLs to the 'urls' list
    urls = urls + img_urls

# Modify the URLs to replace 'smph' with 'bgph' and 'psp' with 'pbp' in the image path
j = 1
list=[]
for url in urls:
    url = url.replace("smph", "bgph")
    url = url.replace("psp", "pbp")
    if url.endswith('.jpg') and url.startswith('/produitsedr'):
      list.append(url)
    if j == 200:
      break
    j += 1

# Open a file to save the links
links = open('/content/links.txt', 'w')

# Write the modified URLs to the file, only those ending with '.jpg' and starting with '/produitsedr'
for url in list:
  links.write('https://www.voursa.com' + url + '\n')

# Close the file after writing all the URLs
links.close()

# Reopen the file to read the links
links = open('/content/links.txt', 'r')

i = 1

!mkdir imgs
# Download each image from the URLs in the file
for url in links.readlines():
    # Open a file to save the image
    with open('/content/imgs/web1_' + str(i) + '.jpg', 'wb') as handle:
        response = requests.get(url.strip(), stream=True)

        if not response.ok:
            print(response)

        # Write the image content to the file in chunks
        for block in response.iter_content(1024):
            if not block:
                break
            handle.write(block)

    i += 1

# Close the links file
links.close()

# Zip the downloaded images
!zip -r /content/imgs.zip /content/imgs/

# Download the zip file containing the images
from google.colab import files
files.download("/content/imgs.zip")
